In [1]:
# extract sit from TOPAZ4b FreeRun for 2000-2010 application

import os
import xarray as xr
import pandas as pd
import numpy as np
import yaml
import pickle as pkl
from datetime import datetime
from datetime import timedelta
from glob import glob

import src.utils.load_config as load_config
from src.data_preparation import load_data
import src.feature_extraction.extract_pca as extract_pca

import src.utils.tardisml_utils as tardisml_utils
rootdir = tardisml_utils.get_rootdir()

In [2]:
# Path to config file
file_config = '../config/config_default_2023.yaml'

nosit_dir, withsit_dir, _, forcing_bdir, pca_dir, res_dir, fig_dir, ml_dir, freerun_dir = load_config.load_filename(file_config)
timeofday, target_field, forcing_fields, covar_fields, lim_idm, lim_jdm, n_comp = load_config.load_config_params(file_config)


# -------------------------------------------------
# selection files

years = [2000, 2011] 
# years = [2011, 2012] 



listfile_a = sorted(glob(os.path.join(rootdir + freerun_dir,'*.nc')))
listyear = [os.path.basename(name)[:4] for name in listfile_a]  # read years
# selection corresponding years
listfile_a = [listfile_a[idx] for idx, name in enumerate(listyear) if int(name)>=years[0] and int(name)<=years[-1]]

# -------------------------------------------------

data_kind = "nosit"
n_components = load_config.get_n_components(data_kind, file_config)

In [3]:
rootdir

'/scratch/project_465000269/edelleo1/'

In [4]:
freerun_dir

'Leo/Jiping_2023/TP4b_freerun/'

In [5]:
listfile_a[-3:]

['/scratch/project_465000269/edelleo1/Leo/Jiping_2023/TP4b_freerun/20111229_dm-12km-NERSC-MODEL-TOPAZ4B-ARC-RAN.fv2.0.nc',
 '/scratch/project_465000269/edelleo1/Leo/Jiping_2023/TP4b_freerun/20111230_dm-12km-NERSC-MODEL-TOPAZ4B-ARC-RAN.fv2.0.nc',
 '/scratch/project_465000269/edelleo1/Leo/Jiping_2023/TP4b_freerun/20111231_dm-12km-NERSC-MODEL-TOPAZ4B-ARC-RAN.fv2.0.nc']

In [6]:
# load Topaz 4b Free Run
nc_sel_na, chrono_na = extract_pca.load_TOPAZ(listfile_a[:20], target_field=target_field, lim_idm=lim_idm, lim_jdm=lim_jdm)

Define chronology from .nc files...
Loading .nc ...
Variable selection...
Spatial selection...


In [8]:
import datetime

In [9]:
# if extend_3m:
first_oct = datetime.datetime(years[0], 12, 1)
last_march = datetime.datetime(years[-1], 3, 31)
idx_start = 200
idx_end = 3000

#nc_sel_na = nc_sel_na.isel(time=slice(idx_start, idx_end))

In [14]:
chrono_na

,date
0,2000-01-01
1,2000-01-02
2,2000-01-03
3,2000-01-04
4,2000-01-05
5,2000-01-06
6,2000-01-07
7,2000-01-08
8,2000-01-09
9,2000-01-10


In [15]:
np.where(chrono_na==datetime.datetime(2000, 1, 12))[0][0]

11

In [17]:
nca_sel_na_ext = nc_sel_na.isel(time=slice(11,14))

In [18]:
nca_sel_na_ext

<xarray.DataArray 'sithick' (time: 3, y: 479, x: 450)>
dask.array<getitem, shape=(3, 479, 450), dtype=float32, chunksize=(1, 479, 450), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2000-01-12 2000-01-13 2000-01-14
    latitude   (y, x) float32 dask.array<chunksize=(479, 450), meta=np.ndarray>
    longitude  (y, x) float32 dask.array<chunksize=(479, 450), meta=np.ndarray>
  * x          (x) float32 -25.5 -25.38 -25.25 -25.12 ... 30.25 30.38 30.5 30.62
  * y          (y) float32 -36.25 -36.12 -36.0 -35.88 ... 23.12 23.25 23.38 23.5
Attributes:
    cell_methods:   area: mean where sea_ice
    grid_mapping:   stereographic
    standard_name:  sea_ice_thickness
    units:          m

In [32]:
# import pdb; pdb.set_trace()

# load pca from 2010-2020
# --------- TOPAZ4b FREERUN ------------------
data_kind = "nosit"
n_components = load_config.get_n_components(data_kind, file_config)
filename = os.path.join(rootdir,pca_dir,f"pca_{target_field}_{n_components}N_noSITass_2011_2019_FreeRun.pkl")
pca_na = load_data.load_pca(filename)

/LUMI_TYKKY_KyB8utx/env1/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [33]:
# apply PCA of 2010-2020 to 2000-2010

Xna = nc_sel_na

maskok = (np.isfinite(Xna)).all(dim='time')
maskok1d = maskok.stack(z=('y','x')).compute()
PCs_na = extract_pca.pca_to_PC(pca_na, Xna, maskok1d)


KeyboardInterrupt: 

In [ ]:
str_na = 'FreeRun'  # 'noSITass' or 'FreeRun'

In [ ]:
# save pca for 2000-2010 as .pkl file

filename = f'pca_{target_field}_TOPAZ4b23_{years[0]}_{years[-1]}_{str_na}.pkl'
extract_pca.save_pca(filename, PCs_na)

# extract_pca.save_pca(filename, pca)
#print(f'PCA saved: {filename}')
#pkl.dump(pca, open(filename,"wb"))